In [11]:
import re

In [41]:
text = "2. \n\xe2\x80\x9csayaaaaang\xe2\x80\x9d\n\xe2\x80\x9ckamu dimana?\xe2\x80\x9d\n\xe2\x80\x9cby\xe2\x80\x9d 123456789x\n\xe2\x80\x9cudah makan?\xe2\x80\x9d\n\xe2\x80\x9caku takut diajak minum\xe2\x80\x9d\n\xe2\x80\x9csebatang boleh?\xe2\x80\x9d\n\xe2\x80"

In [42]:
text = re.sub(r'(\\x[0-9a-fA-F]{2})+', '', text)
text = re.sub(r'[!"$%&\'()*+,\-./:;<=>?@\[\\\]^{|}~_#]', ' ', text)
text = re.sub(r'\s+', ' ', text).strip()
print(text)

2 âsayaaaaangâ âkamu dimana â âbyâ 123456789x âudah makan â âaku takut diajak minumâ âsebatang boleh â â
